# DSSATTools Example notebook

In this notebook you'll see the basic usage of DSSATools library to:
1. Create the inputs
2. Initialize the simulation environment
3. Run simulations
4. Modify input objects
5. Read outputs.

The idea of this library is to be a low-code interface to DSSAT

In [1]:
from DSSATTools import (Crop, SoilProfile, Weather, Management, DSSAT, TabularSubsection)
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [2]:
# Random weather data
DATES = pd.date_range('2008-01-01', '2010-12-31')
N = len(DATES)
df = pd.DataFrame(
    {
    'tn': np.random.gamma(10, 1, N),
    'rad': np.random.gamma(10, 1.5, N),
    'prec': [0.0]* N,
    'rh': 100 * np.random.beta(1.5, 1.15, N),
    },
    index=DATES,
)
df['TMAX'] = df.tn + np.random.gamma(5., .5, N)
# Create a WheaterStation instance
wth = Weather(
    df, 
    {'tn': 'TMIN', 'TMAX': 'TMAX', 'prec': 'RAIN', 
     'rad': 'SRAD', 'rh': 'RHUM'}, 4.3434237,-74.3606715, 1800
)
wth

Weather data at -74.361°, 4.343°
  Date start: 2008-01-01
  Date end: 2010-12-31
Average values:
TMAX    12.641100
TMIN    10.125392
RAIN     0.000000
SRAD    15.072809
RHUM    57.643069
dtype: float64

## Creating Weather, Soil and Crop instances

In [3]:
# Soil instance from default soil profile
soil = SoilProfile(default_class='SIC')

# Crop
crop = Crop('Potato')

/home/diego/Py_DSSATTools/DSSATTools/crop.py:224: UserWarning: No cultivar was indicated, default cultivar IB0003 will be used
  warnings.warn(


## Defining irrigation schedule

In [9]:
schedule = pd.DataFrame([
    (datetime(2008,1,15), 80),
    (datetime(2008,1,30), 60),
    (datetime(2008,2,15), 40),
    (datetime(2008,3,1),  20)
], columns = ['date', 'IRVAL'])
schedule['IDATE'] = schedule.date.dt.strftime('%y%j')
schedule['IROP'] = 'IR001' # Code to define the irrigation operation
schedule = schedule[['IDATE', 'IROP', 'IRVAL']]
schedule

,IDATE,IROP,IRVAL
0,08015,IR001,80
1,08030,IR001,60
2,08046,IR001,40
3,08061,IR001,20


## Create a management instance

In [10]:
# Management instance
man = Management(
    planting_date=DATES[10],
)
# Modify harvest to Maturity
man.simulation_controls['HARVS'] = 'M'
# Set the irrigation schedule
man.irrigation['table'] = TabularSubsection(schedule)
# Set irrigation control as reported schedule
man.simulation_controls['IRRIG'] = 'R'

# The definition of this parameters is mandatory for roots
man.planting_details['PLWT'] = 1500
man.planting_details['SPRL'] = 2

In [11]:
dssat = DSSAT()
dssat.setup()
dssat.run(
    soil=soil, weather=wth, crop=crop, management=man,
)
# Run and check the final Yield

/tmp/dssatduyhkmfw created.
Static files copied to /tmp/dssatduyhkmfw.
RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 PT   1  29 -99 10206  9193     0   200   315     0     0   -99      0  206


In [12]:
# Modify the irrigation amounts and check output
schedule['IRVAL'] = [20, 20, 20, 20]
man.irrigation['table'] = TabularSubsection(schedule)
dssat.run(
    soil=soil, weather=wth, crop=crop, management=man,
)

RUN    TRT FLO MAT TOPWT HARWT  RAIN  TIRR   CET  PESW  TNUP  TNLF   TSON TSOC
           dap dap kg/ha kg/ha    mm    mm    mm    mm kg/ha kg/ha  kg/ha t/ha
  1 PT   1  29 -99  3136  2461     0    80   202     0     0   -99      0  206


In [13]:
# Terminate the DSSAT instance
dssat.close()

/tmp/dssatduyhkmfw and its content has been removed.
